In [1]:
import pandas as pd
import json
import numpy as np
import pickle
from tqdm.notebook import tqdm

import Levenshtein as lev
# https://www.datacamp.com/community/tutorials/fuzzy-string-python
from fuzzywuzzy import fuzz

In [18]:
#ascent_df = pickle.load(open('./df_ascent.pkl',mode='rb'))
ascent_df = pickle.load(open('./df_ascent.pkl',mode='rb'))
len(ascent_df)

3742986

In [11]:
crags = ascent_df['crag'].unique()
crags, len(crags)

([Railay, Sjöända, Gubben i berget, Tjörnbroklippan, Seglora, ..., Raek, Dolina Nielepicka, Zmeinaya gora, Datça, Dolina Czubrowicka]
 Length: 1029
 Categories (1029, object): [Railay, Sjöända, Gubben i berget, Tjörnbroklippan, ..., Dolina Nielepicka, Zmeinaya gora, Datça, Dolina Czubrowicka],
 1029)

In [4]:
df = pd.read_json('west_and_texas.json')
#df = pd.read_json('all_states.json')

In [5]:
df[df.area_name == 'Smith Rock']

,state,area_name,id,latitude,longitude,elevation
220,Oregon,Smith Rock,/area/105788989/smith-rock,44.36765,-121.13889,"2,644"


In [6]:
df

,state,area_name,id,latitude,longitude,elevation
0,California,Wagon Caves,/area/105737114/wagon-caves,36.08410,-121.40100,"1,522"
1,California,Tar Creek (the Swimming Hole),/area/105796891/tar-creek-the-swimming-hole,34.45730,-118.92320,737
2,Oregon,P-IKE Boulders,/area/105984561/p-ike-boulders,45.37150,-123.26170,240
3,Oregon,Oceansides/Netarts,/area/118293499/oceansidesnetarts,45.46653,-123.97443,2
4,Oregon,Cannon Beach,/area/118665560/cannon-beach,45.86307,-123.96478,3
...,...,...,...,...,...,...
702,California,Highway 38 Crags,/area/109292743/highway-38-crags,34.14625,-116.98327,"5,741"
703,California,Galena Peak,/area/111678100/galena-peak,34.07121,-116.84611,"7,400"
704,California,Big Bear Lake Area,/area/109292716/big-bear-lake-area,34.25471,-116.93604,"6,745"
705,California,South San Diego County,/area/111860940/south-san-diego-county,32.74169,-116.88568,618


In [17]:
distdict = {}

for area in tqdm(df['area_name']):
    cragdict = {}
    for crag in crags:
        cragdict[crag] = lev.distance(area,crag)
    
    closestmatch = min(cragdict, key=cragdict.get)
    closestlev = lev.distance(area,closestmatch)
    
    if closestlev <= 3:
        print(closestlev,':  ',area,'---',closestmatch)
        distdict[area] = closestmatch

3 :   Paddock --- Waldeck
2 :   The Orb --- The Dry
3 :   Iron Mountain --- Stone Mountain
3 :   The VC --- The Pit
3 :   Falling Rock --- Hanging Rock
3 :   Sunshine --- Fusine
3 :   Ice Box --- Ice Pond
3 :   The Danks --- The Peaks
3 :   Thor Peak --- The Peaks
2 :   Ice Point --- Ice Pond
3 :   Hidden Falls --- Hidden Valley
3 :   Lightning Creek --- Lightner Creek
3 :   Iron Creek --- Groom Creek
2 :   Crow --- Oro
3 :   Vex Wall --- T Wall
3 :   Saco Crag --- Earl Crag
3 :   Bryant Ledge --- Grand Ledge
0 :   Pine Mountain --- Pine Mountain
0 :   Lion's Head --- Lion's Head
0 :   Sugarloaf --- Sugarloaf
0 :   Table Mountain --- Table Mountain
2 :   Mud Mountain --- 2nd Mountain
3 :   Spy Rock --- Big Rock
3 :   Raven Rocks --- Red Rocks
0 :   Sawmill --- Sawmill
3 :   Deers Leap --- Lovers Leap
3 :   Carter Ledge --- Carter Lake
3 :   Iron Mountain --- Stone Mountain
3 :   Cave Mountain --- Pine Mountain
3 :   Echo Crag --- Earl Crag
2 :   Lost Ledge --- Rose Ledge
3 :   Found Le

In [14]:
len(distdict)

117

In [12]:
distdict = {}

for area in tqdm(df['area_name']):
    cragdict = {}
    for crag in crags:
        cragdict[crag] = fuzz.token_sort_ratio(area,crag)
    
    closestmatch = max(cragdict, key=cragdict.get)
    part_rat = fuzz.token_sort_ratio(area,closestmatch)
    
    if part_rat > 75:
        print(part_rat,':  ',area,'---',closestmatch)
        distdict[area] = closestmatch

78 :   Tower Rock --- Rocktown
80 :   Ozone --- Boone
78 :   Jailhouse Rock --- Jailhouse
100 :   Tuolumne Meadows --- Tuolumne Meadows
78 :   Onion Creek --- Indian creek
77 :   Cumberland Crags --- Cumberland
76 :   McLoughlin Canyon --- Logan Canyon
86 :   Burge Mountain --- Blue mountains
78 :   Crackenstein --- Merkenstein
80 :   Cowiche Canyon --- Chek Canyon
77 :   Palouse Falls --- Taylors Falls
77 :   Mount Rainier --- Pine Mountain
78 :   Monster Rock --- Mortar rock
82 :   Dog Canyon --- Logan Canyon
95 :   Smith Rock --- Smith Rocks
84 :   Coethedral --- Cathedral
86 :   Spring Mountain --- Pine Mountain
76 :   Eagle Rock --- Castle Rock
77 :   Bandon --- Andonno
87 :   Canyon Creek --- Chek Canyon
100 :   Santa Barbara --- Santa Barbara
77 :   Fourth Canyon --- Cougar Canyon
88 :   The Point --- The Pit
78 :   Glen Canyon --- Maple Canyon
81 :   Castle Rock Area --- Castle Rock
85 :   Ring Mountain --- Pine Mountain
78 :   The Tramway --- Tramway
80 :   Halo Boulder --- Ho

In [21]:
fuzz.partial_ratio('Smith rock','smith rocks')

90

In [19]:
area = 'Smith Rock'
cragdict = {}
for crag in crags:
    cragdict[crag] = fuzz.token_sort_ratio(area,crag)

In [20]:
sorted(cragdict, key=cragdict.get, reverse=True)

['Smith Rocks',
 'Focksta',
 'Yosemite',
 'Rocktown',
 'Blacksmith Fork',
 'Red Rocks',
 'Rocklands',
 'Donner summit',
 'Shelf Road',
 'St Géry',
 'Flock Hill',
 'Sítio do Rod',
 'Roccamorice',
 'St Léger',
 'Roaches',
 'Kusieta',
 'Ith',
 'Malibu Creek',
 'Kronthal',
 'Squamish',
 'Cromlech',
 'The Mine',
 'Sandrock',
 'Bronkies',
 'Mt Charleston',
 'City of Rocks',
 "Jack's Canyon",
 'Okic',
 'Kårstø',
 'omis',
 'Omis',
 'Montes Claros',
 'Dolina Czubrowicka',
 'Mt Lemmon',
 'Black Mountain',
 'Lost Cove',
 'Mt Coolum',
 'Red Stone',
 'Boat Rock',
 'Kangaroo Point',
 'Massacre Rocks',
 'Cedar Mtn, Cody',
 'Erikstein',
 'Grochowiec',
 'Krzemionki',
 'St Antonin',
 'Horsetooth',
 'Smugglers Notch',
 'St Cezaire',
 'Governor Stable',
 'Cocalzinho',
 'Ostas',
 'Nassereith',
 'Ouro preto',
 'Stone Fort',
 'Meio Mango',
 'Südlicher Frankenjura',
 'Arcos',
 'Arboli',
 'Priest Draw',
 'Grotti',
 'Kis-Gerecse',
 'Czarnorzeki',
 'Castle Hill',
 'Castle Rock',
 'Riglos',
 'Kochel',
 'Mortar ro

In [ ]:
sdf = pd.Data